In [1]:
import platform, torch, subprocess
print("Host:", platform.node())               # gorina6
print("CUDA:", torch.cuda.is_available())     # True
print(subprocess.getoutput("nvidia-smi | head -n 5"))

Host: gorina6
CUDA: True
Fri Oct  3 14:24:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.03             Driver Version: 550.144.03     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |


In [2]:
import torch, platform
print("Host:", platform.node())
print("Torch:", torch.__version__)
print("CUDA (build):", torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())
print("cuDNN:", torch.backends.cudnn.version())
print("GPU count:", torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print(i, torch.cuda.get_device_name(i))

Host: gorina6
Torch: 2.7.1+cu118
CUDA (build): 11.8
CUDA available: True
cuDNN: 90100
GPU count: 8
0 Tesla V100-PCIE-32GB
1 Tesla V100-PCIE-32GB
2 Tesla V100-PCIE-32GB
3 Tesla V100-PCIE-32GB
4 Tesla V100-PCIE-32GB
5 Tesla V100-PCIE-32GB
6 Tesla V100-PCIE-32GB
7 Tesla V100-PCIE-32GB


In [3]:
import torch
free, total = torch.cuda.mem_get_info()
print(f"GPU mem free/total: {free/1e9:.2f} / {total/1e9:.2f} GB")

GPU mem free/total: 15.83 / 34.07 GB


In [4]:
import torch
print(torch.cuda.get_device_name(0))

Tesla V100-PCIE-32GB


In [13]:
import os, torch
print("CUDA_VISIBLE_DEVICES =", os.environ.get("CUDA_VISIBLE_DEVICES"))
print("Visible count =", torch.cuda.device_count())
print("Using:", torch.cuda.get_device_name(0))

CUDA_VISIBLE_DEVICES = None
Visible count = 8
Using: Tesla V100-PCIE-32GB


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"


In [3]:
import torch
torch.cuda.device_count()  # should be 1

1

In [4]:
from pathlib import Path
import pandas as pd
metadata_csv = Path("/home/stud/fwag/bhome/ele670_project/data/raw/metadata.csv")
dataset_root = Path("/home/stud/fwag/bhome/ele670_project/data/raw/labelled_images")

df = pd.read_csv(metadata_csv, sep=";")
df["finding_class"] = df["finding_class"].astype(str).str.strip()
df["filename"] = df["filename"].astype(str).str.strip()
# Build relative image_path that matches your folder layout
# labelled_images/<finding_class>/<filename>
df["image_path"] = df.apply(
    lambda r: str(Path(r["finding_class"]) / r["filename"]), axis=1
)

bad = df[~df["image_path"].apply(lambda p: (dataset_root / p).exists())]
print(f"Missing: {len(bad)} / {len(df)}")
print(bad.head())

Missing: 786 / 47248
                       filename          video_id  frame_number  \
6865  eb0203196e284797_1157.jpg  eb0203196e284797          1157   
6866  eb0203196e284797_1158.jpg  eb0203196e284797          1158   
6867  eb0203196e284797_1160.jpg  eb0203196e284797          1160   
6868  eb0203196e284797_1167.jpg  eb0203196e284797          1167   
6869  eb0203196e284797_1168.jpg  eb0203196e284797          1168   

     finding_category     finding_class     x1     y1     x2     y2     x3  \
6865          Anatomy  Ampulla of Vater  238.0  196.0  327.0  196.0  327.0   
6866          Anatomy  Ampulla of Vater  138.0    0.0  251.0    0.0  251.0   
6867          Anatomy  Ampulla of Vater   69.0    0.0  153.0    0.0  153.0   
6868          Anatomy  Ampulla of Vater    4.0  115.0   56.0  115.0   56.0   
6869          Anatomy  Ampulla of Vater   57.0  182.0  137.0  182.0  137.0   

         y3     x4     y4                                  image_path  
6865  289.0  238.0  289.0  Ampulla 

In [5]:
from pathlib import Path
import pandas as pd

dataset_root = Path("/home/stud/fwag/bhome/ele670_project/data/raw/labelled_images")

# Build case-insensitive map of class folders that exist on disk
classes_on_disk = {p.name.lower().strip(): p.name for p in dataset_root.iterdir() if p.is_dir()}

# Which CSV class names don't have a direct case-insensitive match?
csv_classes = sorted(df["finding_class"].astype(str).str.strip().unique())
no_match = [c for c in csv_classes if c.lower().strip() not in classes_on_disk]
print(f"[INFO] Class names with no folder match (case-insensitive): {len(no_match)}")
print(no_match[:20])  # peek

# Optional: add manual fixes if needed (e.g., punctuation variants)
manual_map = {
    # "CSV value": "Exact folder name on disk"
    # e.g., "Reduced Mucosal View": "Reduced mucosal view",
    # add entries if you see them in `no_match`
}

def resolve_class_folder(name: str) -> str:
    name = str(name).strip()
    if name in manual_map:
        return manual_map[name]
    return classes_on_disk.get(name.lower().strip(), name)  # fallback to original (will show as missing if wrong)

# Rebuild image_path using the resolved folder name
df["image_path"] = df.apply(lambda r: str(Path(resolve_class_folder(r["finding_class"])) / r["filename"]), axis=1)

# Check again
bad = df[~df["image_path"].apply(lambda p: (dataset_root / p).exists())]
print(f"[CHECK] Missing after folder mapping: {len(bad)} / {len(df)}")
bad.head(10)

[INFO] Class names with no folder match (case-insensitive): 0
[]
[CHECK] Missing after folder mapping: 0 / 47248


,filename,video_id,frame_number,finding_category,finding_class,x1,y1,x2,y2,x3,y3,x4,y4,image_path


In [6]:
# Save a cleaned / normalized CSV for training

out_csv = Path("/home/stud/fwag/bhome/ele670_project/data/processed/metadata_normalized.csv")

# Pick only the columns you actually need for training
df_out = df[["image_path", "finding_class", "video_id"]].copy()

# Optional: add the encoded label column (integer codes for classes)
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
df_out["encoded_label"] = lb.fit_transform(df_out["finding_class"])

df_out.to_csv(out_csv, index=False)

print(f"[SAVED] Normalized metadata CSV → {out_csv}")
print(df_out.head())

[SAVED] Normalized metadata CSV → /home/stud/fwag/bhome/ele670_project/data/processed/metadata_normalized.csv
                                       image_path        finding_class  \
0  Normal clean mucosa/0728084c8da942d9_22803.jpg  Normal clean mucosa   
1  Normal clean mucosa/0728084c8da942d9_22804.jpg  Normal clean mucosa   
2  Normal clean mucosa/0728084c8da942d9_22805.jpg  Normal clean mucosa   
3  Normal clean mucosa/0728084c8da942d9_22806.jpg  Normal clean mucosa   
4  Normal clean mucosa/0728084c8da942d9_22807.jpg  Normal clean mucosa   

           video_id  encoded_label  
0  0728084c8da942d9              9  
1  0728084c8da942d9              9  
2  0728084c8da942d9              9  
3  0728084c8da942d9              9  
4  0728084c8da942d9              9  


In [7]:
# Number of unique classes per video
per_video_classes = df.groupby("video_id")["finding_class"].nunique().sort_values(ascending=False)

print("[INFO] How many classes each video has (top 10):")
print(per_video_classes.head(10))

# Distribution of "number of classes per video"
print("\n[INFO] Distribution of class counts per video:")
print(per_video_classes.value_counts().sort_index())

# Example: list which classes appear together in a specific video
example_video = per_video_classes.index[0]  # video with most classes
classes_in_example = df.loc[df["video_id"] == example_video, "finding_class"].unique()
print(f"\n[INFO] Classes found in video {example_video}:")
print(classes_in_example)

[INFO] How many classes each video has (top 10):
video_id
04a78ef00c5245e0    6
48579eec79784294    6
64440803f87b4843    6
8ebf0e483cac48d6    6
8885668afb844852    5
5e59c7fdb16c4228    5
eb0203196e284797    5
3ada4222967f421d    4
131368cc17e44240    4
7a47e8eacea04e64    4
Name: finding_class, dtype: int64

[INFO] Distribution of class counts per video:
finding_class
1     2
2     8
3    18
4     8
5     3
6     4
Name: count, dtype: int64

[INFO] Classes found in video 04a78ef00c5245e0:
['Pylorus' 'Normal clean mucosa' 'Ileocecal valve' 'Lymphangiectasia'
 'Angiectasia' 'Blood - fresh']


In [8]:
n_videos = df["video_id"].nunique()
print(f"Total distinct videos: {n_videos}")

Total distinct videos: 43


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

import torch
print("CUDA visible devices:", torch.cuda.device_count())
print("Using GPU:", torch.cuda.get_device_name(0))

CUDA visible devices: 1
Using GPU: Tesla V100-PCIE-32GB


In [2]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as T

# ==== CONFIG ====
CSV_PATH = "/bhome/fwag/ele670_project/data/processed/metadata_normalized.csv"
IMG_ROOT = "/bhome/fwag/ele670_project/data/raw/labelled_images"
BATCH_SIZE = 8
NUM_WORKERS = 4

# ==== DATASET ====
class KvasirDataset(Dataset):
    def __init__(self, csv_path, img_root, transform=None):
        self.df = pd.read_csv(csv_path)
        self.img_root = img_root
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_root, row["image_path"])
        img = Image.open(img_path).convert("RGB")
        label = int(row["encoded_label"])
        
        if self.transform:
            img = self.transform(img)
        return img, label

# Simple transforms for sanity check
transform = T.Compose([
    T.Resize((224,224)),
    T.ToTensor()
])

dataset = KvasirDataset(CSV_PATH, IMG_ROOT, transform)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

# ==== MODEL ====
num_classes = dataset.df["encoded_label"].nunique()
model = models.resnet50(weights=None)   # no pretrained, just sanity check
model.fc = nn.Linear(model.fc.in_features, num_classes)

# ==== SANITY TRAIN LOOP (1 batch only) ====
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

images, labels = next(iter(loader))
images, labels = images.to(device), labels.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# forward
outputs = model(images)
loss = criterion(outputs, labels)

# backward
optimizer.zero_grad()
loss.backward()
optimizer.step()

print(f"Sanity check done ✅ | Batch size: {images.size(0)}, Loss: {loss.item():.4f}")

Sanity check done ✅ | Batch size: 8, Loss: 3.3002


In [ ]:
# ==== Tiny ResNet50 sanity trainer for Kvasir-Capsule ====
# - Beginner-friendly, step-by-step
# - Prints debug info along the way
# - Optimized for Tesla V100 (mixed precision, pinned memory, etc.)
# =========================================================

import os, time
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision.models as models

from sklearn.model_selection import GroupShuffleSplit
from collections import Counter

# ---------------- CONFIG ----------------
CSV_PATH = "/bhome/fwag/ele670_project/data/processed/metadata_normalized.csv"
IMG_ROOT   = "/bhome/fwag/ele670_project/data/raw/labelled_images"
IMG_SIZE   = 224
BATCH_SIZE = 32     # safe for V100 (can go higher if memory allows)
NUM_WORKERS = 4     # adjust depending on system
EPOCHS     = 2      # keep small for quick sanity check
LR         = 1e-3
SEED       = 42

# ----------------------------------------
torch.manual_seed(SEED)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
if device == "cuda":
    print(torch.cuda.get_device_name(0))
    print("Memory allocated:", round(torch.cuda.memory_allocated(0)/1024**3, 2), "GB")

# --------------- DATASET ----------------
class KvasirDataset(Dataset):
    """Custom dataset to read image paths + labels from CSV."""
    def __init__(self, df, img_root, transform):
        self.df = df.reset_index(drop=True)
        self.img_root = img_root
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        # Join root with relative path from CSV
        img_path = os.path.join(self.img_root, row["image_path"])
        try:
            img = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"[WARNING] Failed to open {img_path}: {e}")
            # use black image as fallback
            img = Image.new("RGB", (IMG_SIZE, IMG_SIZE), (0,0,0))
        img = self.transform(img)
        label = int(row["encoded_label"])
        return img, label

# --------------- LOAD CSV ----------------
df = pd.read_csv(CSV_PATH, sep=",")
print("First rows of metadata_processed.csv:")
print(df.head())

# How many unique classes?
num_classes = df["encoded_label"].nunique()
print(f"Number of classes: {num_classes}")

# --------------- TRAIN/VAL SPLIT ----------------
# Keep videos separated (no leakage between train/val)
gss = GroupShuffleSplit(n_splits=1, train_size=0.8, random_state=SEED)
train_idx, val_idx = next(gss.split(df, groups=df["video_id"]))
df_train, df_val = df.iloc[train_idx], df.iloc[val_idx]

print(f"Train samples: {len(df_train)} | Val samples: {len(df_val)}")
print(f"Train videos: {df_train['video_id'].nunique()} | Val videos: {df_val['video_id'].nunique()}")

# --------------- TRANSFORMS ----------------
# Simple transforms for sanity check
mean = (0.485, 0.456, 0.406)
std  = (0.229, 0.224, 0.225)

train_tf = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.RandomHorizontalFlip(p=0.5),
    T.ToTensor(),
    T.Normalize(mean, std),
])

val_tf = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.ToTensor(),
    T.Normalize(mean, std),
])

# --------------- DATA LOADERS ----------------
train_ds = KvasirDataset(df_train, IMG_ROOT, train_tf)
val_ds   = KvasirDataset(df_val,   IMG_ROOT, val_tf)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=NUM_WORKERS, pin_memory=True)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=True)

print("One training batch (images, labels):")
sample_imgs, sample_labels = next(iter(train_loader))
print("Image batch shape:", sample_imgs.shape)
print("Label batch shape:", sample_labels.shape)

# --------------- MODEL ----------------
# Load ResNet50 with ImageNet weights (helps it train faster)
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
model.fc = nn.Linear(model.fc.in_features, num_classes)  # replace classifier

model = model.to(device)

# Loss + optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# Mixed precision (faster on V100)
scaler = torch.cuda.amp.GradScaler()

# --------------- TRAINING LOOP ----------------
def run_epoch(loader, train=True):
    if train:
        model.train()
    else:
        model.eval()

    total_loss, total_correct, total_samples = 0.0, 0, 0

    for step, (imgs, labels) in enumerate(loader, start=1):
        imgs = imgs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        with torch.cuda.amp.autocast(enabled=True):
            outputs = model(imgs)
            loss = criterion(outputs, labels)

        if train:
            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

        # Accuracy calculation
        preds = outputs.argmax(dim=1)
        correct = (preds == labels).sum().item()
        total_correct += correct
        total_loss += loss.item() * labels.size(0)
        total_samples += labels.size(0)

        # Print debug info every 10 steps
        if step % 10 == 0 or step == 1:
            print(f"  Step {step}/{len(loader)} | Loss: {loss.item():.4f} | "
                  f"Batch acc: {correct/labels.size(0):.2f}")

    avg_loss = total_loss / total_samples
    avg_acc = total_correct / total_samples
    return avg_loss, avg_acc

# --------------- TRAIN FOR A FEW EPOCHS ----------------
print("\n==== START TRAINING ====\n")
best_val_acc = 0.0
start_time = time.time()

for epoch in range(1, EPOCHS+1):
    print(f"\nEpoch {epoch}/{EPOCHS}")

    train_loss, train_acc = run_epoch(train_loader, train=True)
    print(f"Train: loss={train_loss:.4f}, acc={train_acc:.3f}")

    val_loss, val_acc = run_epoch(val_loader, train=False)
    print(f"Val:   loss={val_loss:.4f}, acc={val_acc:.3f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "tiny_resnet50_best.pt")
        print(f"  ✅ New best model saved (acc={best_val_acc:.3f})")

print("\nTraining finished!")
print("Best validation accuracy:", round(best_val_acc, 3))
print("Elapsed time: %.1f sec" % (time.time() - start_time))

Using device: cuda
Tesla V100-PCIE-32GB
Memory allocated: 0.38 GB
First rows of metadata.csv:
                                       image_path        finding_class  \
0  Normal clean mucosa/0728084c8da942d9_22803.jpg  Normal clean mucosa   
1  Normal clean mucosa/0728084c8da942d9_22804.jpg  Normal clean mucosa   
2  Normal clean mucosa/0728084c8da942d9_22805.jpg  Normal clean mucosa   
3  Normal clean mucosa/0728084c8da942d9_22806.jpg  Normal clean mucosa   
4  Normal clean mucosa/0728084c8da942d9_22807.jpg  Normal clean mucosa   

           video_id  encoded_label  
0  0728084c8da942d9              9  
1  0728084c8da942d9              9  
2  0728084c8da942d9              9  
3  0728084c8da942d9              9  
4  0728084c8da942d9              9  
Number of classes: 14
Train samples: 37430 | Val samples: 9818
Train videos: 34 | Val videos: 9
One training batch (images, labels):
Image batch shape: torch.Size([32, 3, 224, 224])
Label batch shape: torch.Size([32])

==== START TRAINING 

/tmp/ipykernel_3190418/2432252680.py:126: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
/tmp/ipykernel_3190418/2432252680.py:141: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):


  Step 1/1170 | Loss: 2.7401 | Batch acc: 0.00
  Step 10/1170 | Loss: 0.9073 | Batch acc: 0.75
  Step 20/1170 | Loss: 0.5375 | Batch acc: 0.84
  Step 30/1170 | Loss: 1.3675 | Batch acc: 0.69
  Step 40/1170 | Loss: 1.3706 | Batch acc: 0.62
  Step 50/1170 | Loss: 0.9294 | Batch acc: 0.75
  Step 60/1170 | Loss: 0.6208 | Batch acc: 0.78
  Step 70/1170 | Loss: 1.0341 | Batch acc: 0.59
  Step 80/1170 | Loss: 0.5050 | Batch acc: 0.88
  Step 90/1170 | Loss: 0.6382 | Batch acc: 0.84
  Step 100/1170 | Loss: 0.4670 | Batch acc: 0.84
  Step 110/1170 | Loss: 0.3199 | Batch acc: 0.94
  Step 120/1170 | Loss: 0.6109 | Batch acc: 0.78
  Step 130/1170 | Loss: 0.3193 | Batch acc: 0.94
  Step 140/1170 | Loss: 0.5663 | Batch acc: 0.81
  Step 150/1170 | Loss: 0.4441 | Batch acc: 0.84
  Step 160/1170 | Loss: 0.6226 | Batch acc: 0.81
  Step 170/1170 | Loss: 0.3706 | Batch acc: 0.94
  Step 180/1170 | Loss: 0.4049 | Batch acc: 0.97
  Step 190/1170 | Loss: 0.4931 | Batch acc: 0.84
  Step 200/1170 | Loss: 0.1632 